In [8]:
# Import necessary packages
import requests
import time
from mastodon import Mastodon
from dotenv import load_dotenv
import os
import re
import json
from hdfs import InsecureClient

# Create an instance of mastodon
mastodon = Mastodon(
    client_id=os.getenv('client_id'),
    client_secret=os.getenv('client_secret'),
    access_token=os.getenv('access_token'),
    api_base_url='https://mastodon.social'
)

my_account = mastodon.account_verify_credentials()

In [24]:
# Initialize an empty list
mastodon_data = []

while True:
    # Retrieve the last id from last_id_file
    last_id_file = 'last_id.txt'
    with open(last_id_file, 'r') as f:
        last_id = f.read()

    # Replace id by none if not exists
    if not last_id:
        last_id = None

    # Retrieve data from mastodon api
    currently_data = mastodon.timeline_public(limit=40, since_id=last_id)

    # Put the last retrieved id in the last_id file
    with open(last_id_file, 'w') as f:
        f.write(str(currently_data[-1]['id']))
    
    # Extend the data 
    mastodon_data.extend(currently_data)

    print(f"The length of the data is {len(mastodon_data)}", end='\r')

    if len(mastodon_data) >= 4000:
        break


In [6]:
# Define parameters
params = {
    'limit': 40
}

# Request the api
response = requests.get('https://mastodon.social/api/v1/timelines/public', params)

mastodon_data = []
if (response.status_code) == 200:
    actual_data = response.json()
mastodon_data.extend(actual_data)

list_links = []

# Establish a connection to HDFS
client = InsecureClient('http://localhost:9870', user='hadoop')

while(True):

    # Retrieve next link
    next_link = re.search(r'<(.*?)>; rel="next"', response.headers['link']).group(1)

    if next_link in list_links:
        break

    list_links.append(next_link)

    try:
        # Fetch next link
        response = requests.get(next_link, params)

        # Process if status is 200 and get the new data
        if (response.status_code) == 200:
            actual_data = response.json()
        mastodon_data.extend(actual_data)

        # Print the current length
        print(f"The len of the data is {len(mastodon_data)}", end='\r')

        # Break if len reached
        if len(mastodon_data) > 1000:
            break
    except: 
        # Break in errors occurs
        print('Failed to fetch next link')
        break


# Transform data into a JSON file
file_path = 'mastodon_data.json'
with open(file_path, 'w') as file:
    json.dump(mastodon_data, file)

# Load the JSON file into HDFS
client.upload('/hadoop/data/raw/mastodon_data.json', file_path)

# End of process
print('End of process')

End of process data is 1040
